# 9 - Secondary Species Shells:

### In this tutorial we will learn about secondary species shells within Architector with 3 examples:

**(A)** Adding water solvent molecules to a generated Fe\(H<sub>2</sub>O\)<sub>6</sub> molecular complex with automated sampling.

**(B)** Increasing configurational sampling with \[BF<sub>4</sub>\]<sup>-</sup> counterions around the Fe\(H<sub>2</sub>O\)<sub>6</sub> molecular complex.

**(C)** Building non-covalently bound "clusters" of Uranyl (UO<sub>2</sub>) compounds. 

## Starting from **(A)**: 
We will use many of the same imports from previous tutorials!

In [ ]:
from architector import (build_complex,
                         view_structures,
                         convert_io_molecule)
import time # Time is used to track how long each of the cells take!

In [ ]:
# First, we build the [Fe(H2O)6]2+ compound for a time reference!
start = time.time()
inp0 = {'core':{'metal':'Fe','coreType':'octahedral'}, # Make it octahedral
      'ligands':['water']*6, # Give 6 waters
      'parameters':{}}
out0 = build_complex(inp0)
end = time.time()
print('Total time: {} seconds'.format(end-start))

In [ ]:
# Looks good. On my laptop this took only 3 seconds.
view_structures(out0)

In [ ]:
# Now, let's add more waters in the secondary solvation shell:
start = time.time()
inp1 = {'core':{'metal':'Fe','coreType':'octahedral'}, # Make it octahedral
      'ligands':['water']*6, # Give 6 waters
      'parameters':{"add_secondary_shell_species":True, # Switch on secondary shell species
                    "species_list":['water']*3, # Add 3 more waters to outside of iron complex
                   }}
out1 = build_complex(inp1)
end = time.time()

In [ ]:
# Since the previous cell will output a bunch of excess information, let's do the printing here:
print('Total time: {} seconds'.format(end-start))
view_structures(out1)

## Let's check the spin and charge of the new complex:

In [ ]:
mol = convert_io_molecule(out1[list(out1.keys())[0]]['mol2string'])
print('Unpaired Electrons',mol.uhf)
print('Molecule Charge',mol.charge)

## Great, looks like 3 additional waters were added to the outside of the iron complex with automated sampling and the spin/charge makes sense! 

On my laptop this only added an additional 3 seconds of time to the sampling.

## Now, onto  **(B)**: 
We want to add \[BF<sub>4</sub>\]<sup>-</sup> counterions around the Fe\(H<sub>2</sub>O\)<sub>6</sub> molecular complex, and perform additional sampling of the secondary solvation sphere!

The smiles string for BF4- is: "\[B-\](F)(F)(F)F".

In [ ]:
start = time.time()
inp2 = {'core':{'metal':'Fe','coreType':'octahedral'},
      'ligands':['water']*6,
      'parameters':{"add_secondary_shell_species":True, # Switch it on
                   "species_list":['[B-](F)(F)(F)F']*2, # The BF4- ions can be specified via smiles strings!
                   "species_location_method":"default", # This can be set to "random" to random location addition.
                    # By default a colomb-like approach is used!
                   "species_add_copies":3, # Now, we can do 3 random trials of secondary species addition
                   }}
out2 = build_complex(inp2)
end = time.time()

In [ ]:
print('Total time: {} seconds'.format(end-start))
view_structures(out2)

### Looks good! But what about the spin/charge of the system, and where did the other sampled configurations go?

First, the spin/charge of the system:

In [ ]:
mol = convert_io_molecule(out2[list(out2.keys())[0]]['mol2string'])
print('Unpaired Electrons',mol.uhf)
print('Molecule Charge',mol.charge)

### Looks good! With the two counterions the metal center spin should be the same and the charge is set to 0, which is what we'd expect.

Now, the shells:

In [ ]:
# Here the sampled solvation shells were returned as architector molecules with energies attached!
sampled_molecules = out2[list(out2.keys())[0]]['sampled_solvation_shells']
sampled_energies = [x.param_dict['energy'] for x in sampled_molecules]
view_structures(sampled_molecules,labels=sampled_energies)

### With keen eyes, you should be able to tell that the lowest-energy conformer is the one returned as default by Architector as output!

# Now, the real power of this functionality can be shown with **(C)**

Here we can string together actinide metal centers!

In [ ]:
# First, build the UO2 starting point desired!
inp3 = {'core':{'metal':'U','coreCN':4},
      'ligands':['oxo']*2+['hydroxyl']*2, # Generate a neutral UO2(OH)2 complex
      'parameters':{'metal_ox':6}} # Using Ox=6, but can shift to 5 if desired.
out3 = build_complex(inp3)

In [ ]:
# Save this as an initial configuration to add stepwise to other UO2 configurations
structure = out3[list(out3.keys())[0]]['mol2string']

In [ ]:
# This block takes around 20 seconds on my laptop
start = time.time()
inp4 = {'core':{'metal':'U','coreCN':2},
      'ligands':['oxo']*2, # Generate [UO2]2+ actinyl
      'parameters':{
          'metal_ox':6, # Ox for only the initial molecule generated
          'add_secondary_shell_species':True, # Add the docking routine
          'species_grid_rad_skin':0.7, # Reduce to allow for closer molecules
          'species_list':[structure]*4, # Specify how many local copies to add
          'n_species_rotations':100, # Specify number of rotations to try.
          "species_add_copies":10, # Number "randomized" generations to do.
          "species_location_method":'random', # Use random for actiniyls 
          "species_relax":False, # Don't relax (none of the methods are super accurate)
          "species_intermediate_relax":False, # Don't relax intermediate
          "species_intermediate_method":'UFF', # Use UFF for speed
          "species_method":'UFF', # Use UFF for speed
#           "debug":True, # Turn on if you want to see what it's doing in the background 
      }}
out4 = build_complex(inp4)
end = time.time()

In [ ]:
print('Total time: {} seconds'.format(end-start))
view_structures(out4)

### Looks good! How about the spin/charge?

In [ ]:
mol = convert_io_molecule(out4[list(out4.keys())[0]]['mol2string'])
print('Unpaired Electrons',mol.uhf)
print('Molecule Charge',mol.charge)

### Looks good! With only one \[UO<sub>2</sub>\]<sup>2+</sup> ion and only U(VI) oxidation states the spin state should be 0 and the charge should be +2!

How about the other shells we generated?

In [ ]:
view_structures(out4[list(out4.keys())[0]]['sampled_solvation_shells'])

### Looks okay! Since we have to rely on UFF for energetics, and experimental bond distances for actinyls some of these might look a bit funky! Either way, we can generate structures good enough for starting positions.

# 9 - Secondary Species Shells Summary:

### In this tutorial we learned about secondary species shells with Architector with 3 examples:

**(A)** Adding water solvent molecules to a generated Fe\(H<sub>2</sub>O\)<sub>6</sub> molecular complex with automated sampling.

**(B)** Increasing configurational sampling with \[BF<sub>4</sub>\]<sup>-</sup> counterions around the Fe\(H<sub>2</sub>O\)<sub>6</sub> molecular complex.

**(C)** Building non-covalently bound "clusters" of Uranyl (UO<sub>2</sub>) compounds. 